<a href="https://colab.research.google.com/github/danielCangianelli/DesignPatterns/blob/main/Tech_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baixar e preparar o dataset "The AmazonTitles-1.3MM"

In [ ]:
import json

data = []
with open("trn.json", "r", encoding="utf-8") as f:
    for line in f:
        try:
            data.append(json.loads(line.strip()))
        except json.JSONDecodeError:
            print(f"⚠️ Linha com erro ignorada: {line.strip()}")

# Salvar o JSON corrigido
with open("trn_fixed.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4)

print("✅ JSON corrigido salvo em `trn_fixed.json`")

⚠️ Linha com erro ignorada: {"uid": "0061734896", "title": "Obsession: An Erotic Tale", "content": "Vanderbilt&mdash;socialite, artiste and pioneer of designer jeans&mdash;dips into the realm of spanking, thrusting, creaming and opening as a flower toward the sun with this dressed-up pulp erotica tale&mdash;Chip Kidd designed the striking cover, which features white-haired mannequins that (a certain type of person might be inclined to conclude) slightly resemble the host of CNN'sAC360. After renowned architect Talbot Bingham dies, his widow, Priscilla, designates their Maryland estate (of all our estates it was the one I loved most) as a museum. In going through Talbot's papers there, Priscilla discovers letters from a woman calling herself Queen Bee that describe a physical passion utterly foreign to P
✅ JSON corrigido salvo em `trn_fixed.json`


In [ ]:
with open("trn_fixed.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Criar pares pergunta-resposta
fine_tuning_data = []

for item in data:
    question = f"Qual é a descrição do produto {item['title']}?"
    answer = item["content"]
    fine_tuning_data.append({"input": question, "output": answer})

# Salvar dataset formatado
with open("fine_tuning_data.json", "w", encoding="utf-8") as f:
    json.dump(fine_tuning_data, f, indent=4)

print("✅ Dataset formatado e salvo com sucesso!")

✅ Dataset formatado e salvo com sucesso!


# Escolher um modelo base

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Escolher um modelo base (BART ou T5)
model_name = "t5-base"  # ou "t5-small"

# Carregar tokenizador e modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

#  Pré-processar os dados

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.8 MB/s eta 0:00:00


In [ ]:
import json
import html

# Carregar o dataset já formatado
with open("fine_tuning_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Criar um novo dataset com respostas mais estruturadas
fine_tuning_data = []
for item in data:
    description = item["output"]  # ✅ Agora usamos `output`, não `content`

    # Criar um formato mais estruturado para a resposta
    formatted_output = f"The product '{item['input'].split('product ')[-1][:-1]}' is described as: {description}"
    cleaned_output = html.unescape(formatted_output)  # Corrige caracteres especiais

    fine_tuning_data.append({
        "input": item["input"],  # Mantém a pergunta original
        "output": cleaned_output  # Resposta mais estruturada
    })

# Salvar o dataset processado
with open("fine_tuning_data_cleaned.json", "w", encoding="utf-8") as f:
    json.dump(fine_tuning_data, f, indent=4)

print(f"✅ Dataset corrigido salvo com {len(fine_tuning_data)} exemplos!")


✅ Dataset corrigido salvo com 10624 exemplos!


In [ ]:
import json
from datasets import Dataset

# 1️⃣ Carregar o dataset criado
with open("fine_tuning_data_cleaned.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 2️⃣ Converter para Hugging Face Dataset
dataset = Dataset.from_list(data)

# 3️⃣ Reduzir o tamanho do dataset (usar apenas 10.000 exemplos)
dataset = dataset.shuffle(seed=42).select(range(10_000))

# 4️⃣ Dividir em treino (80%) e validação (20%)
dataset = dataset.train_test_split(test_size=0.2)

# 5️⃣ Tokenizar os dados corretamente
def preprocess_data(batch):
    inputs = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=64)
    targets = tokenizer(batch["output"], padding="max_length", truncation=True, max_length=128)

    inputs["labels"] = targets["input_ids"]
    return inputs

# 6️⃣ Aplicar a tokenização
tokenized_dataset = dataset.map(preprocess_data, batched=True)

print("✅ Dados tokenizados com sucesso!")


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

✅ Dados tokenizados com sucesso!


# Treinar o Modelo (Fine-Tuning)

In [7]:
from transformers import TrainingArguments, Trainer

# Configurar os parâmetros do treinamento
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-6,  # Reduzido para aprendizado estável
    per_device_train_batch_size=4,  # Ajuste conforme o hardware
    per_device_eval_batch_size=4,
    num_train_epochs=4,  # Mais épocas para melhor aprendizado
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none",  # Desativar Weights & Biases,
    fp16=True,
)

# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

# Iniciar o treinamento
trainer.train()

print("✅ Fine-Tuning concluído com sucesso!")


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,1.940100,1.809794


Epoch,Training Loss,Validation Loss
1,1.940100,1.809794
2,1.957900,1.770772
3,1.858800,1.754481
4,1.849700,1.749288


✅ Fine-Tuning concluído com sucesso!


# Avaliar Modelo

In [9]:
import torch

def generate_answer(question):
    device = "cuda" if torch.cuda.is_available() else "cpu"  # Detecta GPU ou CPU
    model.to(device)

    # Tokenizar a pergunta e enviar para o mesmo dispositivo do modelo
    inputs = tokenizer(question, return_tensors="pt", padding="max_length", truncation=True, max_length=128).to(device)

    # Gerar resposta usando o modelo
    output = model.generate(**inputs)

    # Decodificar a saída para texto
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Teste antes do treinamento
print("🔹 Testando antes do Fine-Tuning:")
print(generate_answer("Qual é a descrição do produto Girls Ballet Tutu Neon Pink?"))


🔹 Testando antes do Fine-Tuning:
The product 'Qual é a descriço do produto Girls


# Criar o Dataset Estruturado

In [ ]:
import json

# Carregar o dataset original
with open("fine_tuning_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Ver os primeiros exemplos
print("🔹 Primeiros registros do dataset:")
for i in range(5):
    print(data[i])  # ✅ Isso mostra as chaves disponíveis


🔹 Primeiros registros do dataset:
{'input': 'Qual é a descrição do produto Girls Ballet Tutu Neon Pink?', 'output': 'High quality 3 layer ballet tutu. 12 inches in length'}
{'input': 'Qual é a descrição do produto Adult Ballet Tutu Yellow?', 'output': ''}
{'input': 'Qual é a descrição do produto The Way Things Work: An Illustrated Encyclopedia of Technology?', 'output': ''}
{'input': "Qual é a descrição do produto Mog's Kittens?", 'output': 'Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.'}
{'input': 'Qual é a descrição do produto Misty of Chincoteague?', 'output': ''}


# Validar input e output

In [ ]:
import json

# Carregar o dataset criado
with open("fine_tuning_data_cleaned.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Exibir um exemplo aleatório
example = data[0]  # Pegamos o primeiro exemplo
print("🔹 Exemplo do Dataset:")
print(f"Pergunta: {example['input']}")
print(f"Resposta: {example['output']}")

🔹 Exemplo do Dataset:
Pergunta: Qual é a descrição do produto Girls Ballet Tutu Neon Pink?
Resposta: The product 'Qual é a descrição do produto Girls Ballet Tutu Neon Pink' is described as: High quality 3 layer ballet tutu. 12 inches in length


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Tokenizar os dados para o modelo

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

# Carregar o dataset estruturado
with open("fine_tuning_data_cleaned.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Converter para Dataset Hugging Face
dataset = Dataset.from_list(data)

# Reduzir tamanho do dataset para acelerar treino
dataset = dataset.shuffle(seed=42).select(range(5_000))  # Usa só 10k exemplos
dataset = dataset.train_test_split(test_size=0.2)  # 80% treino, 20% validação

# Carregar o tokenizador do modelo
model_name = "t5-small"  # Modelo mais leve para Colab
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenização personalizada
def preprocess_data(batch):
    inputs = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(batch["output"], padding="max_length", truncation=True, max_length=256)

    inputs["labels"] = targets["input_ids"]
    return inputs

# Aplicar tokenização
tokenized_dataset = dataset.map(preprocess_data, batched=False)

print("✅ Dados tokenizados e prontos para treino!")


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

✅ Dados tokenizados e prontos para treino!


In [ ]:
for i in range(3):
    print(f"🔹 Exemplo {i+1} Tokenizado:")
    print(f"Pergunta Tokenizada: {tokenized_dataset['train'][i]['input_ids']}")
    print(f"Resposta Tokenizada (labels): {tokenized_dataset['train'][i]['labels']}\n")


🔹 Exemplo 1 Tokenizado:
Pergunta Tokenizada: [2415, 138, 3, 154, 3, 9, 93, 2685, 8970, 2, 32, 103, 813, 1259, 235, 10520, 1227, 17, 55, 10, 6341, 7504, 7166, 209, 2628, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Resposta Tokenizada (labels): [37, 556, 3, 31, 5991, 138, 3, 154, 3, 9, 93, 2685, 8970, 2, 32, 103, 813, 1259, 235, 10520, 1227, 17, 55, 10, 6341, 7504, 7166, 209, 2628, 31, 19, 3028, 38, 10, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Treinamento do Modelo

In [ ]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

# Carregar o modelo base
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Configuração do treinamento
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-6,
    per_device_train_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
    logging_dir="./logs",
    report_to="none"
)

# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],  # ✅ Usa os 80% do conjunto de treino
    eval_dataset=tokenized_dataset["test"]     # ✅ Usa os 20% do conjunto de validação
)

# Iniciar o treinamento
trainer.train()
print("✅ Modelo treinado com sucesso!")


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,2.826700,2.461430
2,2.513500,2.305597
3,2.345900,2.270122
4,2.371000,2.252704
5,2.333900,2.241387
6,2.295500,2.233752
7,2.290700,2.228175
8,2.235700,2.224670
9,2.266900,2.222538
10,2.304000,2.221834


✅ Modelo treinado com sucesso!


In [19]:
import torch

def generate_answer(question):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    # Adicionar prefixo para o modelo T5 entender a tarefa
    formatted_question = f"question: {question}"

    # Tokenizar entrada
    inputs = tokenizer(formatted_question, return_tensors="pt", padding="max_length", truncation=True, max_length=128).to(device)

    # Gerar resposta
    output = model.generate(
        **inputs,
        max_length=100,
        num_return_sequences=1,
        num_beams=5,
        early_stopping=True
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Teste novamente
print(generate_answer("Qual é a descrição do produto Girls Ballet Tutu Neon Blue?"))


a descriço do produto Girls Ballet Tutu Neon Blue


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "t5-small"  # Modelo base sem treinamento
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

answer = generate_answer("What is the capital of France?")
print(f"🔹 Resposta gerada (modelo base): {repr(answer)}")

import json

# Carregar o dataset treinado
with open("fine_tuning_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Mostrar os primeiros exemplos
print("🔹 Exemplos do dataset de treino:")
for i in range(3):
    print(f"Input: {repr(data[i]['input'])}")  # `repr()` para ver espaços ocultos
    print(f"Output: {repr(data[i]['output'])}\n")


🔹 Resposta gerada (modelo base): 'France'
🔹 Exemplos do dataset de treino:
Input: 'What is the description of the product Girls Ballet Tutu Neon Pink?'
Output: 'High quality 3 layer ballet tutu. 12 inches in length'

Input: "What is the description of the product Mog's Kittens?"
Output: 'Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.'

Input: 'What is the description of the product Girls Ballet Tutu Neon Blue?'
Output: 'Dance tutu for girls ages 2-8 years. Perfect for dance practice, recitals and performances, costumes or just for fun!'

